In [1]:
!pip install transformers torch torchvision torchaudio indic-nlp-library

  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 5.9 MB/s eta 0:00:00
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19171 sha256=3ace575ac1c4e2922d4922bb16c11e1d5278622d8428a121b303921a4495fa6f
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a75a2ffd197a23c8797700ea858b9b34819fbf9e
Successfully built nvidia-ml-py3


In [2]:
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize

## Dataset File

In [4]:
import random
import torch
from copy import deepcopy
from torch.utils.data import Dataset, DataLoader, random_split

hindi_stopwords = {
    "एक", "एवं", "यह", "इस", "के", "का", "की", "को", "में", "है", "हैं", "कर", "किया", "किए", "करते", "करना", "किसी", "गया", "जाता", "जाती", "जाते", "साथ", "अपने", "हुआ", "होता", "होती", "होते", "वाले", "वह", "वहाँ", "जैसा", "जिसका", "जिसकी", "जिसके", "जिनका", "जिनकी", "जिनके", "तथा", "उसके", "उसका", "उसकी", "उनके", "उनका", "उनकी", "उनको", "कुछ", "इसका", "इसकी", "इसके", "सभी", "अगर", "इसमें", "उनका", "उनकी", "उनके", "जैसे", "जिसमें", "तिन्हों", "तिन्हें", "पहले", "बाद", "मानो", "अंदर", "भीतर", "पूरे", "खुद", "आप", "अब", "जब", "जहाँ", "जितना", "जितने", "तब", "वहीं", "हुआ", "होता", "होती", "वाला", "वाली", "वाले"
}

# For Hindi, you can use IndicTokenizer
from indicnlp.tokenize import indic_tokenize

def remove_stopwords(sentence):
    words = indic_tokenize.trivial_tokenize(sentence)
    filtered_words = [word for word in words if word not in hindi_stopwords]
    sentence = ' '.join(filtered_words)
    return sentence
#     sentence = sentence.replace("' '",'"')
#     return filtered_words
#     return ' '.join(filtered_words)


# Example usage
sentence = 'यह एक नमून  वाक्य है जिसमें कुछ शब्दों को हटाया जाएगा।'
filtered_sentence = remove_stopwords(sentence)
print(filtered_sentence)


नमून वाक्य शब्दों हटाया जाएगा ।


In [5]:
class DataAugmentationTechnique:
  ########################################################################
  # Randomly swap the words from sentence
  ########################################################################
    def random_word_swap(self,sentence):
#         words = sentence.split()
        random.shuffle(sentence)
        return sentence
        return ' '.join(words)
    
    def random_deletion(self,words, p):
        if len(words) == 0:
            return words
        if len(words) == 1:
            return words

        #randomly delete words with probability p
        new_words = []
        for word in words:
            r = random.uniform(0, 1)
            if r > p:
                new_words.append(word)

        #if you end up deleting all words, just return a random word
        if len(new_words) == 0:
            rand_int = random.randint(0, len(words)-1)
            return [words[rand_int]]

        return new_words







In [6]:
class HindiPeraphraseDataset:
    def __init__(self, sentences,tokenizer, max_length=100):
        self.sentences = sentences
        self.input_ids = []
        self.attention_mask = []
        self.labels = []
        self.IGNORE_INDEX = -100
        self.tokenizer = tokenizer
        self.dat_obj = DataAugmentationTechnique()
        for sentence in sentences:
            sentence_to_corrupt = deepcopy(sentence)
            sentence_to_delete = deepcopy(sentence)
            for i in range(2):
                corrupted = self.dat_obj.random_word_swap(sentence_to_corrupt)
    #             print(sentence)
    #             print(corrupted)
    #             print()
                sentence_ignore_len = len(self.tokenizer('[BOS]' + " ".join(corrupted) + "[SEP]")["input_ids"])
                encodings = self.tokenizer('[BOS]' + " ".join(corrupted) + "[SEP]" + " ".join(sentence) + '[EOS]', truncation=True,
                                            max_length=max_length, padding="max_length", add_special_tokens=True)
                input_ids = torch.tensor(encodings['input_ids'])
                attention_mask = torch.tensor(encodings['attention_mask'])
                self.input_ids.append(input_ids)
                self.attention_mask.append(attention_mask)

                label = deepcopy(input_ids)
                label[:sentence_ignore_len] = self.IGNORE_INDEX
                self.labels.append(label)
            for i in range(1):
                corrupted = self.dat_obj.random_deletion(sentence_to_delete,0.3)
                if len(corrupted) > 0:
                    sentence_ignore_len = len(self.tokenizer('[BOS]' + " ".join(corrupted) + "[SEP]")["input_ids"])
                    encodings = self.tokenizer('[BOS]' + " ".join(corrupted) + "[SEP]" + " ".join(sentence) + '[EOS]', truncation=True,
                                                max_length=max_length, padding="max_length", add_special_tokens=True)
                    input_ids = torch.tensor(encodings['input_ids'])
                    attention_mask = torch.tensor(encodings['attention_mask'])
                    self.input_ids.append(input_ids)
                    self.attention_mask.append(attention_mask)

                    label = deepcopy(input_ids)
                    label[:sentence_ignore_len] = self.IGNORE_INDEX
                    self.labels.append(label)


    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'labels': self.labels[idx],
            'attention_mask': self.attention_mask[idx]  # unsqueeze to add batch dimension
        }
    
#     def load_data(self):
        

In [7]:
class LoadDataset:
    def __init__(self):
        self.filename = '/kaggle/input/inlp-proj/new.txt'
        self.hindi_stopwords = {
    "एक", "एवं", "यह", "इस", "के", "का", "की", "को", "में", "है", "हैं", "कर", "किया", "किए", "करते", "करना", "किसी", "गया", "जाता", "जाती", "जाते", "साथ", "अपने", "हुआ", "होता", "होती", "होते", "वाले", "वह", "वहाँ", "जैसा", "जिसका", "जिसकी", "जिसके", "जिनका", "जिनकी", "जिनके", "तथा", "उसके", "उसका", "उसकी", "उनके", "उनका", "उनकी", "उनको", "कुछ", "इसका", "इसकी", "इसके", "सभी", "अगर", "इसमें", "उनका", "उनकी", "उनके", "जैसे", "जिसमें", "तिन्हों", "तिन्हें", "पहले", "बाद", "मानो", "अंदर", "भीतर", "पूरे", "खुद", "आप", "अब", "जब", "जहाँ", "जितना", "जितने", "तब", "वहीं", "हुआ", "होता", "होती", "वाला", "वाली", "वाले"
}
        self.sentences = self.read_sentences_from_file()
        

        
    def read_sentences_from_file(self, max_lines=10, lang="hi", remove_nuktas=False):
        sentences = []
        normalizer_factory = IndicNormalizerFactory()
        normalizer = normalizer_factory.get_normalizer(lang, remove_nuktas=remove_nuktas)

        with open(self.filename, encoding="utf-8") as f:
            for _, line in zip(range(4000), f):
                normalized_line = normalizer.normalize(line.strip())
                tokens = indic_tokenize.trivial_tokenize(normalized_line)
                sentences.append(tokens)        
       
        sentences=self.preprocess(sentences)      
        return sentences
    
    
    def preprocess(self, sentences):
        preprocessed_sentences = []
        for sentence in sentences:
            # Replace specific characters
            sentence = [word.replace("’", "").replace("'", "").replace("-", " ") for word in sentence]
            # Remove stopwords
            sentence = [word for word in sentence if word not in self.hindi_stopwords]
            preprocessed_sentences.append(sentence)
        return preprocessed_sentences

    
    def getdata(self):
        return self.sentences

        

In [8]:
ds = LoadDataset()
sentences=ds.getdata()
# ds.preprocess()
# print(len(sentences[8]))

In [9]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/mGPT")
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/mGPT").cuda()
print(len(tokenizer))
tokenizer.add_special_tokens({"sep_token": '[SEP]', "bos_token":'[BOS]'})
tokenizer.pad_token = '[EOS]'
print(tokenizer.all_special_tokens)
model.resize_token_embeddings(len(tokenizer))


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.89M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.45G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


100000
['[BOS]', '<|endoftext|>', '<unk>', '[SEP]', '[EOS]', '<mask>']
GPU memory occupied: 6669 MB.


In [10]:
dataset = HindiPeraphraseDataset(sentences,tokenizer)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
print(len(train_dataset),len(val_dataset))

10798 1200


In [11]:

import gc
gc.collect()
torch.cuda.empty_cache()
# torch.backends.cuda.max_split_size_mb = 0


GPU memory occupied: 5887 MB.


In [12]:
# from transformers import Trainer, TrainingArguments

# training_args = TrainingArguments(learning_rate=5e-6, output_dir='./results', num_train_epochs=3, logging_steps=10, save_steps=5000,optim="adafactor",
#                                   per_device_train_batch_size=1, per_device_eval_batch_size=1,gradient_accumulation_steps=4, gradient_checkpointing=True,
#                                   warmup_steps=10, weight_decay=0.05, logging_dir='./logs', report_to = 'none')
     

In [13]:

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset
# )
# trainer.train()

In [14]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./result',
    num_train_epochs=5,
    logging_steps=10,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=10, 
    weight_decay=0.05,
    optim="adafactor",
    learning_rate=5e-6,
#     warmup_steps=300,  # warmup_steps=gpt_model.num_warmup_steps,
#     weight_decay=0.01,
#         evaluate_during_training=True,
    save_steps=5000,

#         eval_steps=args.save_steps,
#         seed=42,
    logging_dir='./logs',
    gradient_checkpointing=True,
    report_to = 'none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
#         tb_writer=model.writer,
#         prediction_loss_only=True,
)



2024-05-07 20:29:40.837502: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 20:29:40.837630: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 20:29:40.980179: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_con

In [15]:
trainer.train()


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,10.807700
20,7.258400
30,5.205700
40,4.509500
50,6.170600
60,5.805700
70,9.508100
80,6.146400
90,8.784800
100,8.399500


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=13495, training_loss=1.9458285203109011, metrics={'train_runtime': 18689.7337, 'train_samples_per_second': 2.889, 'train_steps_per_second': 0.722, 'total_flos': 3.9144221786112e+16, 'train_loss': 1.9458285203109011, 'epoch': 5.0})

In [16]:
torch.save(model.state_dict(), './model.pth')

In [17]:
from IPython.display import FileLink
FileLink('/kaggle/working/model.pth')


/kaggle/working/model.pth

In [18]:
torch.cuda.empty_cache()


In [19]:
def corrupt(sen):
#     print(sen)
    random.shuffle(sen)
    return sen

In [20]:
def print_paraphrases(sentence):
#     print(sentence)
    normalizer_factory = IndicNormalizerFactory()
    normalizer = normalizer_factory.get_normalizer("hi",remove_nuktas=False)
    corrupted = " ".join(corrupt(indic_tokenize.trivial_tokenize(normalizer.normalize(sentence.strip()))))
    corrupt_in = "[BOS]" + corrupted + "[SEP]"
#     print(corrupt_in)
    generated = tokenizer(corrupt_in, return_tensors="pt").input_ids.cuda()
    sample_outputs = model.generate(generated, do_sample=True, top_k=20,
        top_p=0.975, temperature=1.0,
        no_repeat_ngram_size=10, num_return_sequences=20,max_length=128)
    for i, sample_output in enumerate(sample_outputs):
        print("{}: {}".format(i, tokenizer.decode(sample_output).split("[SEP]")[1].split("[EOS]")[0]))
     

In [21]:
l =["मैं अपनी मां के साथ बाजार के लिए निकला",
   "मैं फल खाता हूँ",
   "मैं चॉकलेट खरीदना चाहता हूं ताकि मैं पढ़ाई कर सकूं",
   "मैं अपने घर जा रहा हूँ",
   "वह बहुत सुन्दर है",
   "वह बड़े ही समय से इस नगर में अपने अध्ययन को समर्पित कर रहा है, और उसकी मेहनत और निष्ठा का परिणाम यह है कि वह अब अपने क्षेत्र में एक प्रमुख नाम बन चुका है।",
   "वह उस नाटक में प्रमुख भूमिका निभाने के लिए बहुत मेहनत कर रहा है।",
   "जीवन में सफलता पाने के लिए प्रयास करो, न कि सफलता की तलाश में भटको। ",
    "वह अपने सपनों को पूरा करने के लिए पूरी ताकत लगाता है।",
    "खुशी वह है जो दूसरों को खुश देखने में मिलती है।",
    "सपने वो नहीं होते जो हम सोते समय देखते हैं, सपने वो होते हैं जो हमें सोने नहीं देते। ",
    "जीवन में सफलता का सबसे बड़ा रहस्य उसमें लगातार मेहनत करना है। ",
    "संघर्ष के बिना कोई भी महान नहीं बन सकता।",
    "संघर्ष के माध्यम से ही हमें अपने असली योग्यता का पता चलता है। ",
    "अपने सपनों को पूरा करने के लिए हमें उनकी महत्ता समझनी चाहिए, न कि उनके दुष्प्रभाव। ",
    "अपने मन का नहीं, अपने कर्मों का नतीजा मिलता है। ",
    "अगर आपका आज परिश्रम से बढ़िया है, तो आपका कल स्वयं आप बेहतर होगा।",
    "कठिनाईयों से डरना नहीं, उनसे लड़ना सीखो।",
    "अपने लक्ष्य को हासिल करने के लिए हमें कभी भी हार नहीं माननी चाहिए।",
    "विश्वास रखें, आशा रखें, और मेहनत करें। ",
    "बदलाव वह नहीं जो हमें डराए, बल्कि बदलाव वह है जिसे हम स्वीकार करते हैं।",
    "सफलता उन्हें मिलती है जो हार नहीं मानते।"]

In [22]:
hindi_stopwords = {
    "एक", "एवं", "यह", "इस", "के", "का", "की", "को", "में", "है", "हैं", "कर", "किया", "किए", "करते", "करना", "किसी", "गया", "जाता", "जाती", "जाते", "साथ", "अपने", "हुआ", "होता", "होती", "होते", "वाले", "वह", "वहाँ", "जैसा", "जिसका", "जिसकी", "जिसके", "जिनका", "जिनकी", "जिनके", "तथा", "उसके", "उसका", "उसकी", "उनके", "उनका", "उनकी", "उनको", "कुछ", "इसका", "इसकी", "इसके", "सभी", "अगर", "इसमें", "उनका", "उनकी", "उनके", "जैसे", "जिसमें", "तिन्हों", "तिन्हें", "पहले", "बाद", "मानो", "अंदर", "भीतर", "पूरे", "खुद", "आप", "अब", "जब", "जहाँ", "जितना", "जितने", "तब", "वहीं", "हुआ", "होता", "होती", "वाला", "वाली", "वाले"
}
for sen in l:
    sen = sen.replace("’", "")
    sen = sen.replace("'", "")
    sen = sen.replace("-", " ")
    words = sen.split()
    filtered_words = [word for word in words if word not in hindi_stopwords]
    sen = ' '.join(filtered_words)
    print()
    print('Original Sentence: ',sen)
    print("Paraphases:" )
    print_paraphrases(sen)


Original Sentence:  मैं अपनी मां बाजार लिए निकला
Paraphases:


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


0: मां मैं मुझे लिए बाजार निकला, बस यहीं मैं बाजार मिली और अपनी मां लोगों से बात करता रहा था ।
1: मुझे मिली बाजार ने मुझे अपनी मां देख लिए कहा मैं क्या चाहकर भी बस हॉट्रिली से नहीं निकलता हूं तो घर लिए बाहर आया ।
2: बाजार अपनी मां निकला और मैं निकला कि मैं वेश्यालय मां कि मैंने भाग लेने नेकि नहीं हूं...
3: मैं अपनी माँ लिए निकला दिन बाजार अपनी बेटी लिए नारा दिए बेटे लाडोस्ट ले लाडोस्ट लिए नारा लगा देता.
4: बाजार अपनी मां तुम लिए राजेश निकला देख मैं बेचारा बाजार नहीं सामना दिलाना चाह पाऊंगा जो सो नहीं होगा ।
5: मां लिए मुझे बाजार निकला बाह्य प्रस्तुत करने मैं अपनी जिंदगी और से ही बौद्धिक स्तर पर गहराई आंखें दिखाई और बाजार से अपना नेतृत्व निकाला.
6: बाजार अपनी मां स्कूल बाजार निकला अपनी फ्रेंड लिए और बाजार ने मैं मां लिए जीवाईनिंग लिए बधाई दी ।
7: मैं अपनी मां लिए बाजार निकला मैं ने मांगलवार अपनी बारे सोचा ।
8: मंगलवार शाम, त्रिमाला मेरे पति ने मेरी दी कि बाजार अपनी मां निकलने लिए मैं नया शासक खड़ा ।
9: मगर अपनी दूसरी दुनिया मैं ही सोच रहा था कि इसे जिंदगी से बहुत तेरी बाजार लिए निकला जा

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

def calculate_contextual_similarity_hindi(sentence1, sentence2):
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
    model = BertModel.from_pretrained('bert-base-multilingual-cased')

    # Tokenize and encode the sentences
    inputs_sentence1 = tokenizer(sentence1, return_tensors='pt', padding=True, truncation=True)
    inputs_sentence2 = tokenizer(sentence2, return_tensors='pt', padding=True, truncation=True)

    # Get BERT embeddings
    with torch.no_grad():
        outputs_sentence1 = model(**inputs_sentence1)
        outputs_sentence2 = model(**inputs_sentence2)

    # Calculate cosine similarity between sentence embeddings
    similarity_score = torch.nn.functional.cosine_similarity(outputs_sentence1.last_hidden_state.mean(dim=1),
                                                             outputs_sentence2.last_hidden_state.mean(dim=1),
                                                             dim=1).item()
    return similarity_score

# Example usage
sentence1 = " सपने वो नहीं होते जो हम सोते समय देखते हैं, सपने वो होते हैं जो हमें सोने नहीं देते।"
sentence2 = "नहीं सपने सपने हमें देते वो हमें सोने देते तो कैसे"
score = calculate_contextual_similarity_hindi(sentence1, sentence2)
print("Contextual similarity score between the Hindi sentences:", score)

In [ ]:
origin_sen = "विश्वास रखें, आशा रखें, और मेहनत करें।" 

pera = ["विश्वास, लिप्त, विश्वास, और मेहनत रखेंगे आशा, युवाओं और कैराकेशन काम करेंगे ।",
"विश्वास, मेहनत और श्रद्धा, आशा, तस्वीरों, विश्वास रखें और करें, मेहनत और करें ।",
"मेहनत और निर्णय लेखन, सुश्रुषिकाओं विश्वास और करें आशा रहेंगी, परिवर्तन से आकर वे रखें, विश्वास ।",
"मेहनत, आशा और मेहनत रखने विश्वास से विजय से बेहतर परंपरा वास्तविक रूप ही नहीं ।",
"आशा होकर, मेहनत और विश्वास रखें, करें चलने और आकर काम करें, तो विश्वास हासिल रखें..",
"यदि जानते ही नामांकन कि विश्वास रखें, करें, विश्वास और मेहनत करें, कौमार्जन करें तो तुम सारी बात ।",
"विश्वास करें, आशा और मेहनत विमान द्वारा अपनी सुविधाओं रखें, लौटने और करेंगी आने विमान द्वारया ।",
"विश्वास रखें, वेतन, करें और मेहनत करें, ज्यादा आशा रखें, और नया शिखर बनाकर आम जनता से करवाए रखें ।",
"मेहनत रखें, आशा, और शुभकामना करें विश्वास वक़्त से दूसरे साथ न करें, शामिल करने और शत्रुघ्न बनकर विश्वास रखें ।",
"", 
"वैसे हमने पूछा, मेहनत किताबें पढ़ाई और विश्वास रखें । विश्वास रखें आशा और मेहमत करें",
"जो संबंधित विश्वास रखें, विश्वास करें, और करें कि आप जीवन मेहनत करें और मेहनत करेंगे ।",
"विश्वास करें कि आपकी सारी मेहनत आशा और मेहनत मेरी होने लगेंगी ।",
"अभी विश्वास, लोग करें, करें, आशा, विश्वास और मेहनत, व्यक्ति ही नहीं बनना और विकास ही तो हो!",
"आशा, ईमान और मेहनत रखें, शानदार और करोड़ों किस्मत सोचते आपने देखा, विश्वास, करें और पैरोड़ी, डॉल्स और स्कूल खा सकते ।",
"आशा, भावनाओं, विश्वास और मेहनत रखें । भोजन बनाते जानवर व्यक्ति, वृक्ष और फूलों जैसी रुचि रखें, लड़कियों आकर्षित करें ।",
"चाहें भलाई, शोषण, मेहनत और बात, करें आशा और विश्वास विचार, करें और मैं जानता हूँ, भगवान ही आरक्षण करें ।",
"मेहनत रखें, आशावादी और मानवतावादी होकर दूसरों से संवाद स्थापित रखें, विश्वास रखें और मिस्टर प्लान बनाए रखें ।",
"भावुक हो और मेहनत रखें, विश्वास रखें और आशा बढ़ाकरें, करें और मेहनत से संतोष रखें ।",
"काम करने विश्वास रखें, और मेहनत करें, लिए मेहनत करते समय रखें आशा और करें, वो ही करें ।"]

print("Original Sentence : ",origin_sen)

for i,sen in enumerate(pera):
    score = calculate_contextual_similarity_hindi(origin_sen,sen)
    print("{}: {} {}".format(i, sen, score))

In [ ]:
origin_sen = "अगर आपका आज परिश्रम से बढ़िया है, तो आपका कल स्वयं आप बेहतर होगा।" 

pera = ["अगर समय हो तो आज बेहतर तो होकर बिना समय खर्च ही आपका परिश्रम बढ़िया स्वयं करिए होगा ।",
"बेहतरीन कल, तो आज आपका स्वयं परिश्रम से आपको बढ़िया होगा आज आपका होगा, कल तो परिश्रम बढ़िया से ।",
"आपका तो कल, आपका होगा और आज ही बढ़िया स्वयं परिश्रम से होगा तो बेहतर बात ।",
"आज का समय बेहतर, आपका होगा परिश्रम आज तो तब बात बेहतर कल आपका स्वयं तो नहीं होगी ।",
"आपका होगा, कल है, तो आज आपका स्वयं बेहतर परिश्रम से आपको बढ़िया आज कल बिताया होगा ।",
"कल है, आपका बेस्ट परिश्रम और होगा स्वयं तो आज आपका ही बेहतर आज तो सुख ही आपका सूख बढ़िया होगा ।",
"आज केदारनिया, कल हो आपका परिश्रम से आपकी स्वयं बढ़िया कल सफलता केदारनिया होगा ।",
"आज होगा तो आपका बेहतर कल, आज मेरा ही स्वयं आज परिश्रम आपका होगा ।",
"कल है, स्वयं आपका बेहतर स्वजन परिश्रम होगा और आपका हजारों से ज्यादा ब्लॉगजगत बढ़िया होगा ।",
"यदि आज स्वयं तो सपनों, क्षितिज, बुन्दे परिश्रम से भी आपका बढ़िया होगा तो बढ़िया परिस्रम ।",
" आपका बेहतर आज, काम करिये तो वो होगा तो होगा आपका तो स्वयं से भी परिश्रम बढ़िया होगा, सारे तीन प्रयत्त करिये ।",
" जब आपका कल है, आपका परिश्रम होगा बेहतर तो स्वयं से ही आज बढ़िया स्वयं करियर प्राप्त होगा ।",
" पहली तो आजकल तो आपका मेहनत से आपका दूसरी बेहतर स्वयं ही परिश्रम होगा ।",
" आज काम स्वयं पाठक होगा तो बेहतर हालातों परिश्रम हाथ से लगा तो आपका होगा बढ़िए, कल स्वयं ही करेंगे ।",
" कल, आपका बेहतर स्वयं कार्य, परिश्रम, आज तो से भविष्य आपका बढ़िया स्वार्थ होगा ।",
" आज आपका समय हास्य स्वयं से आपका होगा, कल कल परिश्रम आपका बढ़िया स्वयँ परिधान करेंगे ।",
" कल है, कल यानि आज आपका वेबसाइट होगा सारा बेहतर ही स्वयं से परिश्रम करेंगे आपका परिचय होगा ।",
" कई बार, आज आपका भविष्य आपका होगा, तो परिश्रम से बेहतर स्वयं होगा..!",
" तो कल है, आपका स्वयं बेहतर आज तो परिवर्तन आपका बेहतर ही से होगा बढ़िया से आपका होगा ।",
" स्वयं होगा आपका आज बेहतर ये होगा कि कल भी आपका परिश्रम से होगा जो ज्यादा ।"]

print("Original Sentence : ",origin_sen)

for i,sen in enumerate(pera):
    score = calculate_contextual_similarity_hindi(origin_sen,sen)
    print("{}: {} {}".format(i, sen, score))